<a href="https://colab.research.google.com/github/rojabugade/IST688_HW1/blob/main/run_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install streamlit PyMuPDF openai
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing instal

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change to the desired directory
import os
os.chdir("/content/drive/MyDrive/IST688")

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
# Print the public IP address
import urllib
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8'))

Password/Endpoint IP for localtunnel is: 34.19.67.75



In [ ]:
# Write the Streamlit app code to app.py
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import fitz  # PyMuPDF for handling PDF files
from openai import OpenAI

# Function to validate and read PDF files using PyMuPDF
def read_pdf(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Show title and description
st.title("📄 RBHomeWorkMate")
st.write(
    "Upload a document below and ask a question about it – GPT will answer! "
)

# Ask user for their OpenAI API key via `st.text_input`
openai_api_key = st.text_input("OpenAI API Key", type="password")
if not openai_api_key:
    st.info("Please add your OpenAI API key to continue.", icon="🗝️")
else:
    try:
        # Create an OpenAI client with the provided API key
        client = OpenAI(api_key=openai_api_key)
    except Exception as e:
        st.error(f"Failed to initialize OpenAI client: {e}")
        st.stop()

    # Let the user upload a file via `st.file_uploader`
    uploaded_file = st.file_uploader(
        "Upload a document (.txt, .md, or .pdf)", type=("txt", "md", "pdf")
    )

    # Ask the user for a question via `st.text_area`
    question = st.text_area(
        "Now ask a question about the document!",
        placeholder="Can you give me a short summary?",
        disabled=not uploaded_file,
    )

    # If a file is uploaded and a question is asked
    if uploaded_file and question:
        try:
            # Read the uploaded file based on its extension
            file_extension = uploaded_file.name.split('.')[-1]
            if file_extension in ['txt', 'md']:
                document = uploaded_file.read().decode()
            elif file_extension == 'pdf':
                document = read_pdf(uploaded_file)
            else:
                st.error("Unsupported file type.")
                st.stop()

            # Prepare the message for GPT model
            messages = [
                {
                    "role": "user",
                    "content": f"Here's a document: {document} {question}",
                }
            ]

            # Generate an answer using the OpenAI API
            stream = client.chat.completions.create(
                model="gpt-4o-mini",  # Ensure the model name is correct
                messages=messages,
                stream=True,
            )

            # Initialize an empty string to accumulate the response
            response_text = ""

            # Iterate through each chunk in the stream
            for chunk in stream:
                # Check if the chunk has choices and delta attributes
                if hasattr(chunk, 'choices') and hasattr(chunk.choices[0], 'delta'):
                    # Extract the delta content safely, ensuring it's a string
                    delta_content = getattr(chunk.choices[0].delta, 'content', '')
                    if delta_content:  # Only concatenate if content is not None
                        response_text += delta_content
                else:
                    st.error("Unexpected response structure.")

            # Display the accumulated response text all at once
            st.write(response_text)

        except Exception as e:
            st.error(f"An error occurred while processing your request: {e}")
    ''')


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



your url is: https://hip-shrimps-share.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.19.67.75:8501

